<a href="https://colab.research.google.com/github/RGivisiez/TensorFlow-Templates/blob/main/TensorBoard_Visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import os

### Model Definitions

In [ ]:
folder_path = './logs1'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

### Initialize Tensorboard

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir $folder_path --host 0.0.0.0

Reusing TensorBoard on port 6006 (pid 168), started 0:46:03 ago. (Use '!kill 168' to kill it.)

### Auxiliary Function

In [ ]:
import os
import tensorflow as tf
from tensorboard.plugins import projector

def Write_Embedding_File(weights, file_path, metadata_file_name):

  '''
     weights: Any variable that can be transformed in a tf.Variable. In this
              case, it will be the values in the embedding layer.

     file_path: Path where the embedding.ckpt will be saved.

     metadata_file_name: Metadata file name.
  '''

  # Save the weights we want to analyze as a variable. Note that the first
  # value represents any unknown word, which is not in the metadata, here
  # we will remove this value.

  weights = tf.Variable(weights)

  # Create a checkpoint from embedding, the filename and key are the
  # name of the tensor.
  checkpoint = tf.train.Checkpoint(embedding=weights)
  checkpoint.save(os.path.join(file_path, "embedding.ckpt"))

  # Set up config.
  config = projector.ProjectorConfig()
  embedding = config.embeddings.add()

  # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
  embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
  embedding.metadata_path = metadata_file_name
  projector.visualize_embeddings(file_path, config)

  return

In [ ]:
import io
import matplotlib.pyplot as plt

def plot_to_image(figure):

  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""

  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')

  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)

  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)

  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

### Write Embedding Files

In [ ]:
import numpy as np

In [ ]:
import string
alphabet = list(string.ascii_lowercase)

In [ ]:
import pandas as pd

#Write a file .tsv with the original values in the
#dataset.

df = pd.DataFrame(np.random.rand(len(alphabet), 3),
                  columns=['feature1', 'feature2', 'feature3'])

df['Names'] = alphabet[:df.shape[0]]
df.set_index('Names', inplace=True)

df.to_csv(folder_path + "/metadata.tsv", sep="\t")

df.tail()

,feature1,feature2,feature3
Names,,,
v,0.175440,0.493535,0.251726
w,0.289284,0.664754,0.787775
x,0.269358,0.373589,0.447067
y,0.592897,0.120832,0.774373
z,0.259286,0.238275,0.745418


In [ ]:
#Write the embedding file.
Write_Embedding_File(np.random.rand(df.shape[0], 10), folder_path,
                     'metadata.tsv')

### Write an Image

In [ ]:
file_writer = tf.summary.create_file_writer(folder_path)

def Image_Plot(model, X, y_hat, epoch):

  pred = model.predict(X)

  # Prepare the plot
  figure = plt.figure()
  plt.scatter(pred, y_hat, c='red')
  plt.plot(y_hat, y_hat, 'k--')
  plt.xlabel('Predictions')
  plt.ylabel('True Values')

  # Convert to image and log
  with file_writer.as_default():
    tf.summary.image("Regression Prediction", plot_to_image(figure), step=epoch)

### Write an Confusion Matrix Image

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
file_writer_cm = tf.summary.create_file_writer(folder_path + '/Confusion_Matrix')

def Confusion_Matrix_Plot(model, X, y_hat, labels_name, epoch=0):
    
    pred = model.predict(X)

    conf = confusion_matrix(y_hat, pred, normalize='true')
    disp = ConfusionMatrixDisplay(conf, display_labels=labels_name)
    disp.plot(cmap='Blues')
    cm_image = plot_to_image(disp.figure_)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)